In [1]:
import pandas as pd
import numpy as np

In [2]:
cwa_path = '../../data/labeled/2018_-_Cities_Water_Actions.csv'
cwr_path = '../../data/labeled/2018_-_Cities_Water_Risks.csv'

In [3]:
cwa = pd.read_csv(cwa_path)
cwa.head()

,Account No,Organisation,City,Country,Region,Access,C40,Reporting Year,Risks to city's water supply,Adaptation action,Action description,City Location
0,1093,City of Atlanta,Atlanta,United States of America,North America,public,NaN,2018,Increased water stress or scarcity,Investment in existing water supply infrastruc...,"The city is building a new reservoir, fixing t...","(33.749, -84.388)"
1,1184,City of Austin,Austin,United States of America,North America,public,C40,2018,Increased water stress or scarcity,Conservation awareness and education,NaN,"(30.2672, -97.7431)"
2,1184,City of Austin,Austin,United States of America,North America,public,C40,2018,Increased water stress or scarcity,Conservation incentives,NaN,"(30.2672, -97.7431)"
3,1184,City of Austin,Austin,United States of America,North America,public,C40,2018,Increased water stress or scarcity,Water use restrictions,NaN,"(30.2672, -97.7431)"
4,1184,City of Austin,Austin,United States of America,North America,public,C40,2018,Increased water stress or scarcity,Efficiency regulations or standards,NaN,"(30.2672, -97.7431)"


In [4]:
cwr = pd.read_csv(cwr_path)
cwr.head()

,Account No,Organisation,City,Country,Region,Access,C40,Reporting year,Risks to city's water supply,Timescale,Magnitude,Risk description,Current population,Population year,City Location
0,1093,City of Atlanta,Atlanta,United States of America,North America,public,NaN,2018,Increased water stress or scarcity,Short-term,Serious,The city depends on one river for water supply...,450182.0,2017,"(33.749, -84.388)"
1,1093,City of Atlanta,Atlanta,United States of America,North America,public,NaN,2018,Higher water prices,Current,Serious,The water cost burden is felt across all ZIP c...,450182.0,2017,"(33.749, -84.388)"
2,1184,City of Austin,Austin,United States of America,North America,public,C40,2018,Increased water stress or scarcity,Medium-term,Extremely serious,NaN,1265974.0,2017,"(30.2672, -97.7431)"
3,1499,Ajuntament de Barcelona,Barcelona,Spain,Europe,public,C40,2018,Increased water stress or scarcity,Current,Extremely serious,The downscaling of the climate projections at ...,1604555.0,2015,"(41.3823, 2.1775)"
4,1499,Ajuntament de Barcelona,Barcelona,Spain,Europe,public,C40,2018,Declining water quality,Medium-term,Serious,A reduction in water resources availability c...,1604555.0,2015,"(41.3823, 2.1775)"


In [5]:
# Convert columns names to lowercase
cwa.columns = cwa.columns.str.lower().str.replace(" ", "_").str.replace("'","")
cwr.columns = cwr.columns.str.lower().str.replace(" ", "_").str.replace("'","")

In [6]:
cwr.isnull().sum()

account_no                       0
organisation                     0
city                             0
country                          0
region                           0
access                           0
c40                            504
reporting_year                   0
risks_to_citys_water_supply     40
timescale                       11
magnitude                      159
risk_description               107
current_population               0
population_year                  0
city_location                    0
dtype: int64

In [7]:
cwa.isnull().sum()

account_no                       0
organisation                     0
city                             0
country                          0
region                           0
access                           0
c40                            536
reporting_year                   0
risks_to_citys_water_supply     75
adaptation_action               19
action_description             146
city_location                    0
dtype: int64

In [8]:
id_columns = list(set(cwr.columns) & set(cwa.columns))
id_columns

['region',
 'account_no',
 'risks_to_citys_water_supply',
 'c40',
 'city',
 'organisation',
 'access',
 'country',
 'reporting_year',
 'city_location']

In [9]:
cwr[id_columns].isnull().sum()

region                           0
account_no                       0
risks_to_citys_water_supply     40
c40                            504
city                             0
organisation                     0
access                           0
country                          0
reporting_year                   0
city_location                    0
dtype: int64

In [10]:
cwa[id_columns].isnull().sum()

region                           0
account_no                       0
risks_to_citys_water_supply     75
c40                            536
city                             0
organisation                     0
access                           0
country                          0
reporting_year                   0
city_location                    0
dtype: int64

In [11]:
cwr['c40'] = cwr['c40'] == 'C40'
cwr[id_columns] = cwr[id_columns].fillna('nan')

In [12]:
cwa['c40'] = cwa['c40'] == 'C40'
cwa[id_columns] = cwa[id_columns].fillna('nan')

In [13]:
len(cwa[id_columns]), len(cwa[id_columns].drop_duplicates())


(685, 484)

In [14]:
len(cwr[id_columns]), len(cwr[id_columns].drop_duplicates())

(608, 578)

In [15]:
cwa_agg = cwa.groupby(id_columns).agg(set).reset_index()
cwr_agg = cwr.groupby(id_columns).agg(set).reset_index()

In [16]:
cwr_agg

,region,account_no,risks_to_citys_water_supply,c40,city,organisation,access,country,reporting_year,city_location,timescale,magnitude,risk_description,current_population,population_year
0,Africa,31146,Inadequate or ageing infrastructure,True,Addis Ababa,Addis Ababa City Administration,public,Ethiopia,2018,"(9.02887, 38.7544)",{Short-term},{Serious},{nan},{3103374.0},{2013}
1,Africa,31146,Increased water demand,True,Addis Ababa,Addis Ababa City Administration,public,Ethiopia,2018,"(9.02887, 38.7544)",{Short-term},{Serious},{nan},{3103374.0},{2013}
2,Africa,31146,Increased water stress or scarcity,True,Addis Ababa,Addis Ababa City Administration,public,Ethiopia,2018,"(9.02887, 38.7544)",{Short-term},{Serious},{nan},{3103374.0},{2013}
3,Africa,31167,Declining water quality,True,Lagos,City of Lagos,public,Nigeria,2018,"(6.52437, 3.3792)",{Medium-term},{Serious},{Increased rainfall will result to greater amo...,{24821418.0},{2017}
4,Africa,31167,Energy supply issues,True,Lagos,City of Lagos,public,Nigeria,2018,"(6.52437, 3.3792)",{Current},{Extremely serious},"{In the occurrence of rainstorms, there will b...",{24821418.0},{2017}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,Southeast Asia and Oceania,54277,Inadequate or ageing infrastructure,False,New Plymouth District,New Plymouth District Council,public,New Zealand,2018,"(-39.1011, 174.354)",{Short-term},{Serious},{Ageing infrastructure such as asbestos piping...,{82130.0},{2017}
574,Southeast Asia and Oceania,54277,Increased water demand,False,New Plymouth District,New Plymouth District Council,public,New Zealand,2018,"(-39.1011, 174.354)",{Short-term},{Serious},{High water consumption per capita means the d...,{82130.0},{2017}
575,Southeast Asia and Oceania,54388,Increased water stress or scarcity,False,Iskandar,Iskandar Regional Development Authority,public,Malaysia,2018,"(5.71447, 100.992)",{Medium-term},{nan},{Projected water supply for both industry and ...,{1900000.0},{2017}
576,Southeast Asia and Oceania,58543,nan,False,Byron Shire,Byron Shire Council,public,Australia,2018,"(-28.6534, 153.533)",{Medium-term},{nan},"{Increased flash flooding is likely, which can...",{31566.0},{2016}


In [17]:
cw_data = pd.merge(cwa_agg, cwr_agg, on=id_columns, how='outer').applymap(lambda x: {np.nan} if pd.isna(x) else x)

In [18]:
cw_data.head()

,region,account_no,risks_to_citys_water_supply,c40,city,organisation,access,country,reporting_year,city_location,adaptation_action,action_description,timescale,magnitude,risk_description,current_population,population_year
0,Africa,31115,nan,True,Johannesburg,City of Johannesburg,public,South Africa,2018,"(-26.2041, 28.0473)",{Water use restrictions},{A by-law that effects water restrictions is i...,{nan},{nan},{nan},{nan},{nan}
1,Africa,31167,Declining water quality,True,Lagos,City of Lagos,public,Nigeria,2018,"(6.52437, 3.3792)",{Water use restrictions},"{Application of water abstraction rules, surch...",{Medium-term},{Serious},{Increased rainfall will result to greater amo...,{24821418.0},{2017}
2,Africa,31167,Energy supply issues,True,Lagos,City of Lagos,public,Nigeria,2018,"(6.52437, 3.3792)",{Other: Alternate Power Source},{Installation of 33KV Solar panel and accesso...,{Current},{Extremely serious},"{In the occurrence of rainstorms, there will b...",{24821418.0},{2017}
3,Africa,31167,Higher water prices,True,Lagos,City of Lagos,public,Nigeria,2018,"(6.52437, 3.3792)",{Water metering},"{Deployment of over 17,000 units prepaid meter...",{Current},{Extremely serious},{Water is highly expensive when compared with ...,{24821418.0},{2017}
4,Africa,31167,Inadequate or aging infrastructure,True,Lagos,City of Lagos,public,Nigeria,2018,"(6.52437, 3.3792)",{Investment in existing water supply infrastru...,{The State Government is currently re-engineer...,{nan},{nan},{nan},{nan},{nan}


In [19]:
cw_data['city_location'] = cw_data['city_location'].apply(eval)
cw_data['latitude'] = cw_data['city_location'].apply(lambda x: x[0])
cw_data['longitude'] = cw_data['city_location'].apply(lambda x: x[1])
cw_data.drop(columns=['city_location'], inplace=True)
id_columns = [x for x in id_columns if x !='city_location'] + ['latitude','longitude']


In [20]:
value_columns = [ x for x in cw_data.columns if x not in id_columns]
len(cw_data[id_columns].drop_duplicates()) # seems that all

677

In [21]:
c40_data = cw_data[cw_data['c40']].copy()

In [22]:
# Values that can be converted back to strings from sets
singular_cols = c40_data.set_index(id_columns).applymap(lambda x: len(x)==1).product() > 0
singular_cols

adaptation_action     False
action_description    False
timescale             False
magnitude              True
risk_description      False
current_population     True
population_year        True
dtype: bool

In [23]:
# magnitude is nicely singular per id
c40_data.loc[:, singular_cols[singular_cols].index] = c40_data.loc[:, singular_cols[singular_cols].index].applymap(lambda x: list(x)[0])

In [24]:
c40_data = c40_data.applymap(lambda x: [t for t in x if not pd.isnull(t) and (t!='ERROR: #NAME?')] if isinstance(x,set) else x)
num_nulls = c40_data[singular_cols[singular_cols].index].isnull()
num_nulls.mean()

magnitude             0.339286
current_population    0.151786
population_year       0.151786
dtype: float64

In [25]:
list_columns = c40_data.iloc[0].apply(lambda x: isinstance(x, list))
list_columns = list_columns[list_columns].index
list_columns

Index(['adaptation_action', 'action_description', 'timescale',
       'risk_description'],
      dtype='object')

In [26]:
list_nulls = c40_data[list_columns].applymap(lambda x: len(x) == 0)
list_nulls.mean()

adaptation_action     0.348214
action_description    0.446429
timescale             0.169643
risk_description      0.321429
dtype: float64

In [27]:
list_nulls.shape, num_nulls.shape

((112, 4), (112, 3))

In [28]:
total_nulls = pd.concat([c40_data[id_columns], list_nulls, num_nulls],axis=1)


In [29]:
c40_data[total_nulls['magnitude']]

,region,account_no,risks_to_citys_water_supply,c40,city,organisation,access,country,reporting_year,adaptation_action,action_description,timescale,magnitude,risk_description,current_population,population_year,latitude,longitude
0,Africa,31115,nan,True,Johannesburg,City of Johannesburg,public,South Africa,2018,[Water use restrictions],[A by-law that effects water restrictions is i...,[],NaN,[],NaN,NaN,-26.20410,28.04730
4,Africa,31167,Inadequate or aging infrastructure,True,Lagos,City of Lagos,public,Nigeria,2018,[Investment in existing water supply infrastru...,[The State Government is currently re-engineer...,[],NaN,[],NaN,NaN,6.52437,3.37920
9,Africa,35863,Inadequate or aging infrastructure,True,Durban,City of Durban,public,South Africa,2018,[Investment in existing water supply infrastru...,[A number of innovative approaches have been p...,[],NaN,[],NaN,NaN,-29.85870,31.02180
14,Africa,35893,Inadequate or aging infrastructure,True,Dar es Salaam,City of Dar es Salaam,public,United Republic of Tanzania,2018,[Investment in existing water supply infrastru...,[City provide some % of the total money correc...,[],NaN,[],NaN,NaN,-6.79235,39.20830
57,East Asia,31111,Inadequate or aging infrastructure,True,Tokyo,Tokyo Metropolitan Government,public,Japan,2018,[],[Renewal of water purification plant],[],NaN,[],NaN,NaN,35.41000,139.41000
62,East Asia,31169,Inadequate or aging infrastructure,True,Hong Kong,Government of Hong Kong Special Administrative...,public,"China, Hong Kong Special Administrative Region",2018,[Other: Other],[Rehabilitation and Replacement (R&R) Programm...,[],NaN,[],NaN,NaN,22.28810,114.14000
68,East Asia,54298,Higher water prices,True,Dalian,Dalian Municipal People's Government,public,China,2018,"[Water metering, Efficiency regulations or sta...",[],[],NaN,[],0.0,0.0,38.93890,121.56700
69,East Asia,54298,Increased water stress or scarcity,True,Dalian,Dalian Municipal People's Government,public,China,2018,"[Water metering, Efficiency regulations or sta...",[],[],NaN,[],0.0,0.0,38.93890,121.56700
74,Europe,1499,Inadequate or aging infrastructure,True,Barcelona,Ajuntament de Barcelona,public,Spain,2018,[Investment in existing water supply infrastru...,[],[],NaN,[],NaN,NaN,41.38230,2.17750
82,Europe,14088,nan,True,Oslo,City of Oslo,public,Norway,2018,[Investment in existing water supply infrastru...,[High activity on re-opening urban rivers and ...,[],NaN,[],NaN,NaN,59.91390,10.75220


In [30]:
c40_data['magnitude'].unique()

array([nan, 'Serious', 'Extremely serious', 'Less Serious'], dtype=object)

In [31]:
c40_risks = c40_data[id_columns+['magnitude']].copy()
c40_risks['low'] = c40_risks['magnitude'] == 'Less Serious'
c40_risks['medium'] = c40_risks['magnitude'] == 'Serious'
c40_risks['high'] = c40_risks['magnitude'] == 'Extremely serious'
c40_risks['unknown'] = c40_risks['magnitude'].isna()
c40_risks = c40_risks.drop(columns=['magnitude']).groupby([x for x in id_columns if x!="risks_to_citys_water_supply"]).agg(sum).reset_index()
c40_risks

,region,account_no,c40,city,organisation,access,country,reporting_year,latitude,longitude,low,medium,high,unknown
0,Africa,31115,True,Johannesburg,City of Johannesburg,public,South Africa,2018,-26.20410,28.04730,0,0,0,1
1,Africa,31146,True,Addis Ababa,Addis Ababa City Administration,public,Ethiopia,2018,9.02887,38.75440,0,3,0,0
2,Africa,31167,True,Lagos,City of Lagos,public,Nigeria,2018,6.52437,3.37920,0,5,2,1
3,Africa,35858,True,Cape Town,City of Cape Town,public,South Africa,2018,-33.92530,18.42390,0,0,1,0
4,Africa,35863,True,Durban,City of Durban,public,South Africa,2018,-29.85870,31.02180,0,0,0,2
5,Africa,35893,True,Dar es Salaam,City of Dar es Salaam,public,United Republic of Tanzania,2018,-6.79235,39.20830,0,2,0,1
6,Africa,35913,True,Nairobi,City of Nairobi,public,Kenya,2018,-1.29206,36.82190,0,2,0,0
7,East Asia,31111,True,Tokyo,Tokyo Metropolitan Government,public,Japan,2018,35.41000,139.41000,0,4,0,1
8,East Asia,31113,True,Yokohama,City of Yokohama,public,Japan,2018,35.44370,139.63800,0,1,0,0
9,East Asia,31169,True,Hong Kong,Government of Hong Kong Special Administrative...,public,"China, Hong Kong Special Administrative Region",2018,22.28810,114.14000,1,1,0,1


In [32]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import sent_tokenize, word_tokenize

In [33]:
import spacy
try:
    nlp = spacy.load("en_core_web_md")
except:
    spacy.cli.download('en_core_web_md')
    nlp = spacy.load("en_core_web_md")

In [34]:
def create_vector(description):
    ret = description.apply(lambda x:nlp(' '.join(x)).vector if x else np.nan)
    replace = ret.mean()
    ret = np.vstack(ret.apply(lambda x: replace if not isinstance(x,np.ndarray) else x))
    return ret

In [35]:
c40_data_action_description = create_vector(c40_data.action_description)
c40_data_adaptation_action = create_vector(c40_data.adaptation_action)

In [36]:
c40_data.risks_to_citys_water_supply = c40_data.risks_to_citys_water_supply.apply(lambda x: np.nan if x=='nan' else x)

In [37]:
c40_data[c40_data.risks_to_citys_water_supply.isnull()]

,region,account_no,risks_to_citys_water_supply,c40,city,organisation,access,country,reporting_year,adaptation_action,action_description,timescale,magnitude,risk_description,current_population,population_year,latitude,longitude
0,Africa,31115,NaN,True,Johannesburg,City of Johannesburg,public,South Africa,2018,[Water use restrictions],[A by-law that effects water restrictions is i...,[],NaN,[],NaN,NaN,-26.2041,28.0473
59,East Asia,31111,NaN,True,Tokyo,Tokyo Metropolitan Government,public,Japan,2018,[],[Implementing measures against flood],[Short-term],Serious,[Flooding],13857664.0,2017.0,35.4100,139.4100
82,Europe,14088,NaN,True,Oslo,City of Oslo,public,Norway,2018,[Investment in existing water supply infrastru...,[High activity on re-opening urban rivers and ...,[],NaN,[],NaN,NaN,59.9139,10.7522
290,North America,14874,NaN,True,"Portland, OR","City of Portland, OR",public,United States of America,2018,[Watershed preservation],[Portland seeks to manage the watershed in way...,[Medium-term],NaN,"[Portland has a secondary, redundant, groundwa...",639863.0,2016.0,45.5200,-122.6820
452,South and West Asia,35904,NaN,True,Kolkata,Kolkata Metropolitan Area,public,India,2018,[Stormwater management (natural or man-made in...,[Channelisation of drains Construction of new ...,"[Short-term, Current]",NaN,[Kolkata is currently ranked as the third most...,14030000.0,2011.0,22.5726,88.3639
456,South and West Asia,50782,NaN,True,Dhaka,Dhaka City,public,Bangladesh,2018,[Stormwater management (natural or man-made in...,[Embankment is constructed around Dhaka City t...,[Short-term],NaN,[Increases water born diseases],4500000.0,0.0,23.8103,90.4125
466,Southeast Asia and Oceania,31166,NaN,True,Jakarta,Jakarta City Government,public,Indonesia,2018,[Other: Stormwater management (natural or man ...,[- Restore damaged and lost water ecosystems (...,[Current],NaN,[Increased frequency of rain affecting the are...,10075310.0,2014.0,-6.1751,106.8650


In [38]:
c40_data[c40_data.risks_to_citys_water_supply.isnull()].action_description.values

array([list(['A by-law that effects water restrictions is in place.']),
       list(['Implementing measures against flood']),
       list(['High activity on re-opening urban rivers and streams and building separate sewage systems.', 'See text on top', 'Small scale project to implement water meetering also in private households.', 'From 2028 connecting a new separate water source to the pipeline network.']),
       list(['Portland seeks to manage the watershed in ways that preserve its health and ecosystem functions.']),
       list(['Channelisation of drains Construction of new drains and drainage pumping station.', 'Complete restriction in use of Ground water. Mandatory Rainwater Harvesting structures for High rises.']),
       list(['Embankment is constructed around Dhaka City to protect flood water.']),
       list(['- Restore damaged and lost water ecosystems (such as developed-over or contaminated lakes, ponds, canals, riversides, estuaries, coasts, mangrove forests, etc.) - Prote

In [39]:
c40_data[c40_data.risks_to_citys_water_supply.isnull()].adaptation_action.values

array([list(['Water use restrictions']), list([]),
       list(['Investment in existing water supply infrastructure', 'Stormwater management (natural or man-made infrastructure)', 'Diversifying water supply (including new sources)', 'Water metering']),
       list(['Watershed preservation']),
       list(['Stormwater management (natural or man-made infrastructure)', 'Efficiency regulations or standards']),
       list(['Stormwater management (natural or man-made infrastructure)']),
       list(['Other: Stormwater management (natural or man made infrastructure)'])],
      dtype=object)

In [40]:
c40_data_adaptation_action

array([[ 0.047787  , -0.00347   , -0.10457667, ..., -0.18500668,
         0.31601   ,  0.33764997],
       [ 0.047787  , -0.00347   , -0.10457667, ..., -0.18500668,
         0.31601   ,  0.33764997],
       [-0.1537648 ,  0.00722   ,  0.126846  , ..., -0.4304208 ,
        -0.041108  ,  0.13713899],
       ...,
       [-0.10501895,  0.05004509,  0.01966717, ..., -0.17703515,
         0.09389435,  0.14128076],
       [-0.10501895,  0.05004509,  0.01966717, ..., -0.17703515,
         0.09389435,  0.14128076],
       [-0.10501895,  0.05004509,  0.01966717, ..., -0.17703515,
         0.09389435,  0.14128076]], dtype=float32)

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
action_description_similarities = [cosine_similarity(nlp('Implementing measures against flood').vector.reshape(1, -1), x.reshape(1, -1)) for x in c40_data_action_description]
adaptation_action_similarities = [cosine_similarity(nlp(' '.join(['Investment in existing water supply infrastructure', 
                                                'Water metering', 'Stormwater management (natural or man-made infrastructure)', 
                                                'Diversifying water supply (including new sources)'])).vector.reshape(1, -1), x.reshape(1, -1)) for x in c40_data_adaptation_action]

In [42]:
c40_data.action_description.iloc[np.argsort(np.array(adaptation_action_similarities).reshape(-1))[::-1]].values

array([list(['High activity on re-opening urban rivers and streams and building separate sewage systems.', 'See text on top', 'Small scale project to implement water meetering also in private households.', 'From 2028 connecting a new separate water source to the pipeline network.']),
       list(["The City is taking steps to supplement our water supplies through groundwater wells, recycled water for irrigation and an aggressive water conservation program. We have also developed guidelines for the use of greywater through the Laundry to Landscape pilot program. Using local water sources reduces the vulnerability that comes from being heavily dependent on distant reservoirs, while at the same time limiting the amount of water we need from the Tuolumne River and keeping our commitment to protect and preserve our watersheds. The Potable Offset Investigation evaluates the â€œmaximum achievableâ€\x9d reduction in Hetch Hetchy Regional Water System demand in San Francisco through the use of o

In [44]:
c40_data.action_description.iloc[np.argsort(np.array(action_description_similarities).reshape(-1))[::-1]].values

array([list(['Implementing measures against flood']),
       list(['Barcelona has the action protocol for risk of drought in the city that aims to prevent and anticipate possible drought situations, establishing preventive measures and defining a model of action for municipal services when there is a drought situation in the city. Specifically, various actions are established according to the scenario (pre-alert, alert, exceptional, emergency I, emergency II, emergency III and recovery), consisting of actions to raise awareness and communicate with the public and concrete actions to save water in services or activities that generate more consumption.']),
       list(['There are a range of actions in place to address the impacts of flooding on water supply, including policies within the Auckland Unitary Plan Operative in Part.The Stormwater Unit has identified the critical parts of the stormwater system, which are vital to providing service continuity and have unacceptable consequences 

In [45]:
c40_data.risks_to_citys_water_supply.unique()

array([nan, 'Declining water quality', 'Energy supply issues',
       'Higher water prices', 'Inadequate or aging infrastructure',
       'Increased water demand', 'Increased water stress or scarcity',
       'Regulatory', 'Inadequate or ageing infrastructure'], dtype=object)

In [46]:
encoder = LabelEncoder()
encoder.fit(c40_data.risks_to_citys_water_supply)
c40_data.risks_to_citys_water_supply_enc = encoder.transform(c40_data.risks_to_citys_water_supply)

<ipython-input-46-7270d861331c>:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  c40_data.risks_to_citys_water_supply_enc = encoder.transform(c40_data.risks_to_citys_water_supply)


In [47]:
c40_data.risks_to_citys_water_supply_enc

array([8, 0, 1, 2, 4, 5, 6, 7, 6, 4, 4, 0, 6, 0, 4, 6, 8, 6, 4, 6, 6, 2,
       6, 0, 4, 5, 6, 5, 6, 0, 6, 6, 8, 4, 6, 0, 6, 6, 6, 4, 6, 4, 6, 6,
       6, 0, 6, 6, 5, 6, 6, 2, 4, 7, 8, 6, 6, 0, 4, 6, 6, 6, 8, 0, 4, 6,
       8, 5, 6, 4, 6, 6, 0, 2, 4, 6, 8, 0, 4, 6, 3, 5, 6, 3, 3, 3, 6, 3,
       3, 3, 7, 0, 3, 3, 1, 3, 0, 6, 7, 3, 3, 3, 2, 3, 5, 7, 0, 2, 3, 5,
       3, 3])

In [48]:
c40_data[c40_data['city']=='Amman']

,region,account_no,risks_to_citys_water_supply,c40,city,organisation,access,country,reporting_year,adaptation_action,action_description,timescale,magnitude,risk_description,current_population,population_year,latitude,longitude
273,Middle East,54337,Increased water demand,True,Amman,Greater Amman Municipality,public,Jordan,2018,[Efficiency regulations or standards],[Miyuhana (the water company which supervises ...,[Current],Extremely serious,[Amman has had a rapidly rising population due...,3800000.0,2017.0,31.9454,35.9284
274,Middle East,54337,Increased water stress or scarcity,True,Amman,Greater Amman Municipality,public,Jordan,2018,"[Use of non-potable water outside, Water meter...",[All water supplies are metered and charged ac...,[Current],Extremely serious,[Jordan is one of the most water-deprived coun...,3800000.0,2017.0,31.9454,35.9284


In [49]:
value_columns

['adaptation_action',
 'action_description',
 'timescale',
 'magnitude',
 'risk_description',
 'current_population',
 'population_year']

In [50]:
to_drop = ['timescale','current_population','population_year']
c40_data_r = c40_data.drop(columns=['timescale','current_population','population_year'])
value_columns = [x for x in value_columns if x not in to_drop] + ['risks_to_citys_water_supply']

In [51]:
vectorized = [create_vector(c40_data_r[x]) for x in value_columns if x in list_columns]

In [52]:
vectorized = np.mean(vectorized,axis=0)


In [55]:
cols_to_encode =  [x for x in value_columns if x not in list_columns]
c40_data_r[cols_to_encode] = c40_data_r[cols_to_encode].fillna('nan')
encoders = {x:LabelEncoder().fit(c40_data_r[x].fillna('nan')) for x in cols_to_encode}

In [56]:
encoded = np.array([encoders[x].transform(c40_data_r[x]) for x in encoders]).T.astype(float)
for t in range(len(cols_to_encode)):
    encoded[encoded[:,t] == np.where(encoders[cols_to_encode[t]].classes_=='nan')[0]] = np.nan

In [57]:
whole = np.hstack([encoded, vectorized])

In [60]:
c40_data[c40_data['city']=='Amman'].action_description.values

array([list(['Miyuhana (the water company which supervises and manages the water distribution to the city) has been putting in place projects to reduce amount of water loss or non-revenue water, which comprises a large percentage of the water supplied.']),
       list(['All water supplies are metered and charged accordingly to encourage reduction in use.', 'Non-potable water is  used for cleaning areas around municipality buildings, and watering plants around the municipality buildings, in parks belonging to the municipality and along the streets within the city.', 'Water is supplied only at certain times of the week, so water supply is constrained by capacity of wells/ tanks for each building.'])],
      dtype=object)

In [61]:
c40_data[c40_data['city']=='Amman'].risk_description.values

array([list(['Amman has had a rapidly rising population due to a relatively high birth rate and the influx of refugees from surrounding countries. This has placed a great deal of stress on an already over-burdened water supply.']),
       list(['Jordan is one of the most water-deprived countries in the world. Long term average rainfall is decreasing gradually in Jordan. At the same time ground water abstraction is above safe yields, since surface sources are insufficient for the population.'])],
      dtype=object)

In [62]:
from sklearn.experimental import enable_iterative_imputer

In [63]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
filled = SimpleImputer(strategy='median').fit_transform(whole)

In [64]:
imputed = np.array([
    encoders[x].inverse_transform(
    filled[:, c].astype(int)
 ) for c,x in enumerate(cols_to_encode)]).T

In [65]:
c40_data_imputed = c40_data_r.copy()

In [66]:
c40_data_imputed[cols_to_encode] = imputed

In [67]:
c40_data_imputed

,region,account_no,risks_to_citys_water_supply,c40,city,organisation,access,country,reporting_year,adaptation_action,action_description,magnitude,risk_description,latitude,longitude
0,Africa,31115,Increased water demand,True,Johannesburg,City of Johannesburg,public,South Africa,2018,[Water use restrictions],[A by-law that effects water restrictions is i...,Serious,[],-26.20410,28.0473
1,Africa,31167,Declining water quality,True,Lagos,City of Lagos,public,Nigeria,2018,[Water use restrictions],"[Application of water abstraction rules, surch...",Serious,[Increased rainfall will result to greater amo...,6.52437,3.3792
2,Africa,31167,Energy supply issues,True,Lagos,City of Lagos,public,Nigeria,2018,[Other: Alternate Power Source],[Installation of 33KV Solar panel and accesso...,Extremely serious,"[In the occurrence of rainstorms, there will b...",6.52437,3.3792
3,Africa,31167,Higher water prices,True,Lagos,City of Lagos,public,Nigeria,2018,[Water metering],"[Deployment of over 17,000 units prepaid meter...",Extremely serious,[Water is highly expensive when compared with ...,6.52437,3.3792
4,Africa,31167,Increased water demand,True,Lagos,City of Lagos,public,Nigeria,2018,[Investment in existing water supply infrastru...,[The State Government is currently re-engineer...,Serious,[],6.52437,3.3792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,Southeast Asia and Oceania,31164,Higher water prices,True,Ho Chi Minh,Ho Chi Minh City,public,Viet Nam,2018,[],[],Less Serious,[],10.76260,106.6600
669,Southeast Asia and Oceania,31164,Inadequate or ageing infrastructure,True,Ho Chi Minh,Ho Chi Minh City,public,Viet Nam,2018,[],[],Less Serious,[],10.76260,106.6600
670,Southeast Asia and Oceania,31164,Increased water demand,True,Ho Chi Minh,Ho Chi Minh City,public,Viet Nam,2018,[],[],Less Serious,[],10.76260,106.6600
671,Southeast Asia and Oceania,31166,Increased water demand,True,Jakarta,Jakarta City Government,public,Indonesia,2018,[],[],Serious,"[Piped water coverage is still very limited, t...",-6.17510,106.8650


In [68]:
pd.concat([c40_data_imputed[cols_to_encode],c40_data_r[value_columns]],axis=1)[c40_data_r[cols_to_encode[0]] == 'nan']

,magnitude,risks_to_citys_water_supply,adaptation_action,action_description,magnitude,risk_description,risks_to_citys_water_supply
0,Serious,Increased water demand,[Water use restrictions],[A by-law that effects water restrictions is i...,nan,[],nan
4,Serious,Increased water demand,[Investment in existing water supply infrastru...,[The State Government is currently re-engineer...,nan,[],Inadequate or aging infrastructure
9,Serious,Increased water demand,[Investment in existing water supply infrastru...,[A number of innovative approaches have been p...,nan,[],Inadequate or aging infrastructure
14,Serious,Increased water demand,[Investment in existing water supply infrastru...,[City provide some % of the total money correc...,nan,[],Inadequate or aging infrastructure
57,Serious,Increased water demand,[],[Renewal of water purification plant],nan,[],Inadequate or aging infrastructure
62,Serious,Increased water demand,[Other: Other],[Rehabilitation and Replacement (R&R) Programm...,nan,[],Inadequate or aging infrastructure
68,Serious,Increased water demand,"[Water metering, Efficiency regulations or sta...",[],nan,[],Higher water prices
69,Serious,Increased water demand,"[Water metering, Efficiency regulations or sta...",[],nan,[],Increased water stress or scarcity
74,Serious,Increased water demand,[Investment in existing water supply infrastru...,[],nan,[],Inadequate or aging infrastructure
82,Serious,Increased water demand,[Investment in existing water supply infrastru...,[High activity on re-opening urban rivers and ...,nan,[],nan


In [69]:
pd.concat([c40_data_imputed[cols_to_encode],c40_data_r[value_columns]],axis=1)[c40_data_r[cols_to_encode[1]] == 'nan']

,magnitude,risks_to_citys_water_supply,adaptation_action,action_description,magnitude,risk_description,risks_to_citys_water_supply
0,Serious,Increased water demand,[Water use restrictions],[A by-law that effects water restrictions is i...,nan,[],nan
59,Serious,Increased water demand,[],[Implementing measures against flood],Serious,[Flooding],nan
82,Serious,Increased water demand,[Investment in existing water supply infrastru...,[High activity on re-opening urban rivers and ...,nan,[],nan
290,Serious,Increased water demand,[Watershed preservation],[Portland seeks to manage the watershed in way...,nan,"[Portland has a secondary, redundant, groundwa...",nan
452,Serious,Increased water demand,[Stormwater management (natural or man-made in...,[Channelisation of drains Construction of new ...,nan,[Kolkata is currently ranked as the third most...,nan
456,Serious,Increased water demand,[Stormwater management (natural or man-made in...,[Embankment is constructed around Dhaka City t...,nan,[Increases water born diseases],nan
466,Serious,Increased water demand,[Other: Stormwater management (natural or man ...,[- Restore damaged and lost water ecosystems (...,nan,[Increased frequency of rain affecting the are...,nan
